In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from collections import defaultdict

In [2]:
df1 = pd.read_csv('D://Documenti//Università//Informatica Umanistica//DMII//occupancy_data//datatraining.txt', skipinitialspace=True, na_values='?', 
                 keep_default_na=True)
df2 = pd.read_csv('D://Documenti//Università//Informatica Umanistica//DMII//occupancy_data//datatest2.txt', skipinitialspace=True, na_values='?', 
                 keep_default_na=True,)

df = pd.concat([df1,df2], ignore_index = True)
pd.to_datetime(df['date'], format='%Y-%m-%d %H:%M:%S')

0       2015-02-04 17:51:00
1       2015-02-04 17:51:59
2       2015-02-04 17:53:00
3       2015-02-04 17:54:00
4       2015-02-04 17:55:00
                ...        
17890   2015-02-18 09:15:00
17891   2015-02-18 09:16:00
17892   2015-02-18 09:16:59
17893   2015-02-18 09:17:59
17894   2015-02-18 09:19:00
Name: date, Length: 17895, dtype: datetime64[ns]

In [3]:
dates = pd.to_datetime(df['date'], infer_datetime_format=True)
df['day'] = dates.dt.day
df['absH'] = dates.dt.hour

In [4]:
df = df.set_index('date')

In [5]:
del df['Humidity']
del df['Light']
del df['HumidityRatio']
del df['Temperature']
del df['Occupancy']

In [6]:
df

,CO2,day,absH
date,,,
2015-02-04 17:51:00,721.25,4,17
2015-02-04 17:51:59,714.00,4,17
2015-02-04 17:53:00,713.50,4,17
2015-02-04 17:54:00,708.25,4,17
2015-02-04 17:55:00,704.50,4,17
...,...,...,...
2015-02-18 09:15:00,1505.25,18,9
2015-02-18 09:16:00,1514.50,18,9
2015-02-18 09:16:59,1521.50,18,9


In [7]:
a = np.array(())

In [8]:
for n in df['day'].unique():
    if n != 4 and n != 10 and n != 11 and n !=18:
        for m in df[df['day'] == n]['absH'].unique():
            b = df[df.day==n][df.absH==m]
            del b['day']
            del b['absH']
            b = b.reset_index(drop=True)
            b = b[:59]
            b = b.to_numpy()
        
            if a.size == 0:
                a = b.copy()
            else:
                a = np.dstack((a,b))

c:\users\gdm17\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


In [9]:
b.shape

(59, 1)

In [10]:
a.shape

(59, 1, 264)

In [11]:
a = a.reshape(264,59,1)

In [12]:
a

array([[[ 456.5       ],
        [ 452.        ],
        [ 440.5       ],
        ...,
        [ 449.        ],
        [ 450.25      ],
        [ 456.5       ]],

       [[ 458.5       ],
        [ 453.        ],
        [ 445.25      ],
        ...,
        [1303.        ],
        [1001.        ],
        [ 762.5       ]],

       [[ 646.5       ],
        [ 546.        ],
        [ 508.        ],
        ...,
        [ 502.5       ],
        [ 508.33333333],
        [ 510.5       ]],

       ...,

       [[ 421.        ],
        [ 425.        ],
        [ 424.        ],
        ...,
        [ 567.        ],
        [ 574.        ],
        [ 583.5       ]],

       [[ 581.5       ],
        [ 584.        ],
        [ 568.        ],
        ...,
        [ 634.33333333],
        [ 630.75      ],
        [ 630.25      ]],

       [[ 654.25      ],
        [ 653.        ],
        [ 658.5       ],
        ...,
        [1512.        ],
        [1412.        ],
        [1202.5       ]]

In [13]:
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from tslearn.piecewise import SymbolicAggregateApproximation

In [14]:
scaler = TimeSeriesScalerMeanVariance(mu=0., std=1.)  # Rescale time series
X = scaler.fit_transform(a)

In [15]:
n_paa_segments = 100
n_sax_symbols = 10
sax = SymbolicAggregateApproximation(n_segments=n_paa_segments, alphabet_size_avg=n_sax_symbols)
ts_sax = sax.fit_transform(X)
sax_ts_inv = sax.inverse_transform(X)

c:\users\gdm17\appdata\local\programs\python\python37\lib\site-packages\tslearn\piecewise.py:163: RuntimeWarning: Mean of empty slice.
  X_transformed[:, i_seg, :] = X[:, start:end, :].mean(axis=1)
c:\users\gdm17\appdata\local\programs\python\python37\lib\site-packages\numpy\core\_methods.py:154: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)
c:\users\gdm17\appdata\local\programs\python\python37\lib\site-packages\tslearn\piecewise.py:35: RuntimeWarning: invalid value encountered in less
  indices = numpy.logical_and(X_symbols < 0, X_paa < bp)


In [16]:
map_symbols = {k: v for v, k in enumerate(np.unique(sax_ts_inv.ravel()))}
map_symbols

{-1.6448536269514729: 0,
 -1.0364333894937898: 1,
 -0.6744897501960817: 2,
 -0.38532046640756773: 3,
 -0.12566134685507402: 4,
 -1.2952855254678513e-231: 5,
 0.12566134685507416: 6,
 0.38532046640756773: 7,
 0.6744897501960817: 8}

In [17]:
X_seq = list()
for x in sax_ts_inv:
    X_seq.append([map_symbols.get(v, -1) for v in x.ravel()])

In [18]:
X_seq[44]

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 2,
 2,
 0,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

# Pattern Mining

In [19]:
from prefixspan import PrefixSpan

In [20]:
ps = PrefixSpan(X_seq)

In [23]:
ps.topk(10, closed=True)

#closed indica che i seguenti pattern non hanno nessun super-insieme con la stessa frequenza

[(264,
  [0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0]),
 (262,
  [0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0]),
 (257, [1]),
 (255,
  [0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0]),
 (253, [2]),
 (248,
  [0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0]),
 (247, [1, 1]),
 (241,
  [0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   

In [33]:
ps.topk(10, generator=True)

[(257, [1]),
 (253, [2]),
 (247, [1, 1]),
 (241, [0, 1]),
 (232, [0, 1, 1]),
 (232, [1, 0]),
 (231, [1, 2]),
 (228, [2, 1]),
 (223, [0, 2]),
 (223, [1, 0, 0])]

In [25]:
ps.frequent(200, closed=True)

[(214, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]),
 (210, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]),
 (264,
  [0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0]),
 (262,
  [0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0]),
 (255,
  [0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0]),
 (248,
  [0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0]),
 (241,
  [0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0

In [26]:
ps.frequent(100, closed=True)

[(214, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]),
 (195, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]),
 (158, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 2]),
 (128, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 2, 1]),
 (102, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 2, 0]),
 (113, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 2, 2]),
 (185, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]),
 (143, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0]),
 (134, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0]),
 (130, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0]),
 (127, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0]),
 (122, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0]),
 (119, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0]),
 (117, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]),
 (114, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]),
 (111, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 (109, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 (

In [32]:
ps.frequent(100, generator=True)

#generator indica che questi pattern non hanno nessun sottoinsieme con la medesima frequenza.

[(241, [0, 1]),
 (232, [0, 1, 1]),
 (202, [0, 1, 1, 2]),
 (146, [0, 1, 1, 2, 0]),
 (140, [0, 1, 1, 2, 0, 0]),
 (149, [0, 1, 1, 2, 2]),
 (108, [0, 1, 1, 2, 2, 0]),
 (102, [0, 1, 1, 2, 2, 0, 0]),
 (195, [0, 1, 1, 0]),
 (185, [0, 1, 1, 0, 0]),
 (211, [0, 1, 2]),
 (184, [0, 1, 2, 1]),
 (151, [0, 1, 2, 1, 0]),
 (145, [0, 1, 2, 1, 0, 0]),
 (124, [0, 1, 2, 1, 1]),
 (103, [0, 1, 2, 1, 1, 0]),
 (153, [0, 1, 2, 0]),
 (147, [0, 1, 2, 0, 0]),
 (158, [0, 1, 2, 2]),
 (120, [0, 1, 2, 2, 1]),
 (114, [0, 1, 2, 2, 0]),
 (108, [0, 1, 2, 2, 0, 0]),
 (214, [0, 1, 0]),
 (101, [0, 1, 0, 2]),
 (205, [0, 1, 0, 0]),
 (223, [0, 2]),
 (199, [0, 2, 1]),
 (167, [0, 2, 1, 0]),
 (161, [0, 2, 1, 0, 0]),
 (142, [0, 2, 1, 1]),
 (115, [0, 2, 1, 1, 0]),
 (111, [0, 2, 1, 1, 0, 0]),
 (171, [0, 2, 0]),
 (164, [0, 2, 0, 0]),
 (170, [0, 2, 2]),
 (132, [0, 2, 2, 1]),
 (104, [0, 2, 2, 1, 0]),
 (123, [0, 2, 2, 0]),
 (117, [0, 2, 2, 0, 0]),
 (257, [1]),
 (247, [1, 1]),
 (213, [1, 1, 2]),
 (157, [1, 1, 2, 0]),
 (151, [1, 1, 2, 0, 0